# Experiment 011: 10-Fold CV + Stronger Regularization

Following evaluator's recommendation to reduce overfitting:
- CV-LB gap is increasing (1.19% → 1.97%) - sign of overfitting
- High fold variance (2.02% spread) suggests instability
- Focus on GENERALIZATION, not maximizing CV

Changes:
1. 10-fold CV (vs 5-fold) for more stable estimates
2. Reduced depth: 6 (vs 8)
3. Increased l2_leaf_reg: 7.0 (vs 3.52)
4. Added subsample: 0.8
5. Added colsample_bylevel: 0.8

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

# Load data
train = pd.read_csv('/home/data/train.csv')
test = pd.read_csv('/home/data/test.csv')
print(f"Train: {train.shape}, Test: {test.shape}")

Train: (8693, 14), Test: (4277, 13)


In [2]:
# Feature engineering (same as best performing experiments)
def feature_engineering(df):
    df = df.copy()
    df['Group'] = df['PassengerId'].apply(lambda x: int(x.split('_')[0]))
    df['PassengerNum'] = df['PassengerId'].apply(lambda x: int(x.split('_')[1]))
    df['Deck'] = df['Cabin'].apply(lambda x: x.split('/')[0] if pd.notna(x) else np.nan)
    df['CabinNum'] = df['Cabin'].apply(lambda x: int(x.split('/')[1]) if pd.notna(x) else np.nan)
    df['Side'] = df['Cabin'].apply(lambda x: x.split('/')[2] if pd.notna(x) else np.nan)
    return df

train = feature_engineering(train)
test = feature_engineering(test)

# GroupSize
all_data = pd.concat([train[['Group']], test[['Group']]], ignore_index=True)
group_sizes = all_data['Group'].value_counts().to_dict()
for df in [train, test]:
    df['GroupSize'] = df['Group'].map(group_sizes)
    df['Solo'] = (df['GroupSize'] == 1).astype(int)
print("Basic features done")

Basic features done


In [3]:
# Group-based imputation
def group_based_imputation(train_df, test_df):
    combined = pd.concat([train_df, test_df], ignore_index=True)
    for col in ['HomePlanet', 'Deck', 'Side']:
        group_mode = combined.groupby('Group')[col].apply(
            lambda x: x.mode()[0] if len(x.mode()) > 0 else np.nan).to_dict()
        for df in [train_df, test_df]:
            mask = df[col].isna()
            df.loc[mask, col] = df.loc[mask, 'Group'].map(group_mode)
    return train_df, test_df

train, test = group_based_imputation(train, test)

# Remaining imputation
def impute_remaining(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    for col in spending_cols:
        mask = (df['CryoSleep'] == True) & (df[col].isna())
        df.loc[mask, col] = 0
    for col in ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].mode()[0])
    for col in ['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'CabinNum']:
        if df[col].isna().any():
            df[col] = df[col].fillna(df[col].median())
    return df

train = impute_remaining(train)
test = impute_remaining(test)
print("Imputation done")

Imputation done


In [4]:
# Create features (same as exp_008 - best feature set)
def create_features(df):
    df = df.copy()
    spending_cols = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    
    df['TotalSpent'] = df[spending_cols].sum(axis=1)
    for col in spending_cols:
        df[f'{col}_ratio'] = df[col] / (df['TotalSpent'] + 1)
        df[f'{col}_spent'] = (df[col] > 0).astype(int)
        df[f'{col}_log'] = np.log1p(df[col])
    
    df['LuxurySpent'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['BasicSpent'] = df['FoodCourt'] + df['ShoppingMall']
    df['LuxuryRatio'] = df['LuxurySpent'] / (df['TotalSpent'] + 1)
    df['SpentPerAge'] = df['TotalSpent'] / (df['Age'] + 1)
    df['SpendingBin'] = pd.cut(df['TotalSpent'], bins=[-1, 0, 500, 2000, float('inf')], labels=[0, 1, 2, 3]).astype(int)
    df['NumSpendingCategories'] = sum(df[f'{col}_spent'] for col in spending_cols)
    df['TotalSpent_log'] = np.log1p(df['TotalSpent'])
    df['LuxurySpent_log'] = np.log1p(df['LuxurySpent'])
    df['BasicSpent_log'] = np.log1p(df['BasicSpent'])
    df['Spa_VRDeck_RoomService'] = df['Spa'] + df['VRDeck'] + df['RoomService']
    df['FoodCourt_RoomService'] = df['FoodCourt'] + df['RoomService']
    
    df['CryoSleep_HomePlanet'] = df['CryoSleep'].astype(str) + '_' + df['HomePlanet'].astype(str)
    df['CryoSleep_Destination'] = df['CryoSleep'].astype(str) + '_' + df['Destination'].astype(str)
    df['Deck_Side'] = df['Deck'].astype(str) + '_' + df['Side'].astype(str)
    df['HomePlanet_Destination'] = df['HomePlanet'].astype(str) + '_' + df['Destination'].astype(str)
    df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 17, 25, 40, 60, 100], 
                            labels=['Child', 'Teen', 'YoungAdult', 'Adult', 'MiddleAge', 'Senior']).astype(str)
    df['AgeGroup_CryoSleep'] = df['AgeGroup'] + '_' + df['CryoSleep'].astype(str)
    df['VIP_HomePlanet'] = df['VIP'].astype(str) + '_' + df['HomePlanet'].astype(str)
    
    df['IsChild'] = (df['Age'] <= 12).astype(int)
    df['IsTeen'] = ((df['Age'] > 12) & (df['Age'] <= 17)).astype(int)
    df['IsYoungAdult'] = ((df['Age'] > 17) & (df['Age'] <= 25)).astype(int)
    df['IsAdult'] = ((df['Age'] > 25) & (df['Age'] <= 60)).astype(int)
    df['IsSenior'] = (df['Age'] > 60).astype(int)
    return df

train = create_features(train)
test = create_features(test)
print("Features created")

Features created


In [5]:
# Encode categorical features with label encoding (best approach)
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side',
                'CryoSleep_HomePlanet', 'CryoSleep_Destination', 'Deck_Side',
                'HomePlanet_Destination', 'AgeGroup', 'AgeGroup_CryoSleep', 'VIP_HomePlanet']

for col in cat_features:
    le = LabelEncoder()
    combined = pd.concat([train[col].astype(str), test[col].astype(str)])
    le.fit(combined)
    train[col + '_enc'] = le.transform(train[col].astype(str))
    test[col + '_enc'] = le.transform(test[col].astype(str))

# Define feature columns (56 features - best feature set)
num_features = [
    'Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'Group', 'PassengerNum', 'CabinNum', 'GroupSize', 'Solo',
    'TotalSpent', 'LuxurySpent', 'BasicSpent', 'LuxuryRatio', 'SpentPerAge', 'SpendingBin',
    'NumSpendingCategories', 'Spa_VRDeck_RoomService', 'FoodCourt_RoomService',
    'RoomService_ratio', 'FoodCourt_ratio', 'ShoppingMall_ratio', 'Spa_ratio', 'VRDeck_ratio',
    'RoomService_spent', 'FoodCourt_spent', 'ShoppingMall_spent', 'Spa_spent', 'VRDeck_spent',
    'RoomService_log', 'FoodCourt_log', 'ShoppingMall_log', 'Spa_log', 'VRDeck_log',
    'TotalSpent_log', 'LuxurySpent_log', 'BasicSpent_log',
    'IsChild', 'IsTeen', 'IsYoungAdult', 'IsAdult', 'IsSenior'
]

feature_cols = [col + '_enc' for col in cat_features] + num_features
X = train[feature_cols].values
y = train['Transported'].astype(int).values
X_test = test[feature_cols].values

print(f"X shape: {X.shape}, Features: {len(feature_cols)}")

X shape: (8693, 56), Features: 56


In [ ]:
# REGULARIZED CatBoost with 10-FOLD CV
cat_params_regularized = {
    'depth': 6,  # Reduced from 8
    'learning_rate': 0.05,
    'iterations': 1000,
    'l2_leaf_reg': 7.0,  # Increased from 3.52
    'subsample': 0.8,  # New - adds randomness
    'colsample_bylevel': 0.8,  # New - adds randomness
    'random_seed': 42,
    'verbose': False
}

# 10-fold CV for more stable estimates
n_folds = 10
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

oof_preds = np.zeros(len(X))
test_preds = np.zeros(len(X_test))
fold_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]
    
    model = CatBoostClassifier(**cat_params_regularized)
    model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=100)
    
    oof_preds[val_idx] = model.predict_proba(X_val)[:, 1]
    test_preds += model.predict_proba(X_test)[:, 1] / n_folds
    
    fold_acc = accuracy_score(y_val, (oof_preds[val_idx] >= 0.5).astype(int))
    fold_scores.append(fold_acc)
    print(f"Fold {fold+1:2d}: {fold_acc:.5f}")

final_acc = accuracy_score(y, (oof_preds >= 0.5).astype(int))
print(f"\n10-Fold Regularized CatBoost CV: {final_acc:.5f} (+/- {np.std(fold_scores):.5f})")

In [ ]:
# Compare with previous experiments
print("\n=== COMPARISON ===")
print(f"This exp (10-fold, regularized): {final_acc:.5f} (+/- {np.std(fold_scores):.5f})")
print(f"exp_008 (5-fold, multi-seed):    0.81698 (+/- 0.00072)")
print(f"exp_003 (5-fold, Optuna):        0.81951 (NOT REPRODUCIBLE)")
print(f"\nFold variance analysis:")
print(f"  Min fold: {min(fold_scores):.5f}")
print(f"  Max fold: {max(fold_scores):.5f}")
print(f"  Range:    {max(fold_scores) - min(fold_scores):.5f}")

# Check if we beat the threshold
print(f"\nTo beat LB 0.8045, need CV > 0.82089")
if final_acc > 0.82089:
    print(">>> SUBMIT THIS! CV exceeds threshold.")
else:
    print(f">>> CV is {0.82089 - final_acc:.5f} below threshold.")

In [ ]:
# Create submission
test_binary = (test_preds >= 0.5).astype(bool)
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Transported': test_binary
})

submission.to_csv('/home/submission/submission.csv', index=False)
print(f"\nSubmission saved")
print(f"CV: {final_acc:.5f}")
print(f"Predicted transported rate: {test_binary.mean():.4f}")
print(f"Training transported rate: {y.mean():.4f}")
print(submission.head())